# Implementação do algoritimo completo, sem compromisso com  perfomance

## Imports

In [1]:
import os
import functools
import operator
import math
import random
import sys

import numpy as np

## Função match

![Match function](./img/match_function.png)

In [2]:
def cluster_matching_function(weight_matrix,
                              cluster_number,
                              element,
                              prototypes,
                              dissimilarity_matrices):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                cluster_number: int
                    Número do cluster em questão
                element: int
                    Índice do elemento (entre 0 e N-1)
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    # Criando aliases compatíveis com os nomes da fórmula
    k = cluster_number
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes[k]
    l = weight_matrix

    dissimilarities_sum = np.array([dj[element, Gk].sum() for dj in D])

    return np.dot(l[k], dissimilarities_sum)

## Função objetivo

![Objetive function](./img/objective_function.png)

In [3]:
def objective_function(clusters_qtd,
                       elements_qtd,
                       adequacy_criterion,
                       m,
                       weight_matrix,
                       prototypes,
                       dissimilarity_matrices):
    """
        :params: clusters_qtd: int
                    Quantidade total de clusters
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação
                weight_matrix:
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    u = np.power(adequacy_criterion, m) # Resolvendo a exponeciação de u de uma vez só
    l = weight_matrix
    D = dissimilarity_matrices
    K = clusters_qtd
    G = prototypes
    N = elements_qtd
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching
  
    J = [sum([u[i, k] * match(l, k, i, G, D) for i in range(N)]) 
          for k in range(K)]


    return sum(J)

## Protótipos

![Prototype function](./img/prototype_function.png)

In [4]:
def get_prototypes(elements_qtd,
                   q,
                   m,
                   s,
                   cluster_number,
                   adequacy_criterion,
                   dissimilarity_matrices,
                   weight_matrix):
    """
        :params:
                elements_qtd: int 
                    Quantidade de elementos da base de dados
                q: int
                    Quantidade de elementos protótipos
                m: int
                    Fator de ponderação do índice de adequação
                s: int
                    Fator de ponderação dos pesos das matrizes
                cluster_number: int
                    Quantidade total de clusters
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                weight_matrix: 
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster

        :return: list

    """

    G = []
    k = cluster_number,
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)
    
    p_sums = np.empty((p, N))
    #print("p_sums", p_sums.shape)
    
    for j in range(p):
        p_sums[j, :] = (D[j] * l[k,j]).sum(axis=1) 
    
    p_sums = p_sums.sum(axis=0)
    #print("p_sums somado", p_sums.shape)
    
    p_sums_rotten = u[:, k].flatten() * p_sums
    
    #print("u[:, k].shape", u[:, k].shape)
    #print("p_sums_pondered", p_sums_pondered.shape)
    
    #print(p_sums_pondered.shape)
    return p_sums_rotten.argsort()[-q:][::-1]
    
#     def dist(element):
#         """
#             Função auxiliar para cálculo da distância de um elemento qualquer 
#             em relação a todos os outros da base de dados, consideran as matrizes 
#             de dissimilaridade e 
#             o critério de adequação

#             return: (int, float)
#                 (element, soma das distâncias)
#         """
#         return element, sum([u[i, k] * sum([l[k, j] * D[j][element, i] for j in range(p)])
#                              for i in range(elements_qtd)])

#     while (len(G) < q):
#         # Calculando todas as distâncias dos elementos que ainda não estão em G
#         distances = [dist(i) for i in range(N) if i not in G]
#         # Obtendo o menor item de distância e separando entre o elemento e sua distância
#         element, _ = min(distances, key=operator.itemgetter(1))
#         G.append(element)

#     return G

## Matriz de relevâcia

![Funções de peso](./img/vector_weights_function.png)

In [5]:
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
    """
        :params:
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array

    """

    D = dissimilarity_matrices
    P = len(D)
    Gk = prototypes
    K = clusters_qtd
    N = elements_qtd
    u = np.power(adequacy_criterion, m)
    l = np.zeros((K, P))

    def match(element, Dh, G):
        """
            Função auxiliar para cálculo de match entre um elemento 
            qualquer, os protótipos G de um cluster específico e uma matriz 
            de similaridade específica Dh.
        """

        #return sum([Dh[element, e] for e in G])
        return Dh[element, G].sum()

    for k in range(K):
        for j in range(P):
            # Calculado o somatório do numerador da equação à esquerda da igualdade
            weight_diss_sum1 = [sum([u[i, k] * match(i, D[h], Gk[k]) for i in range(N)])
                                for h in range(P)]

            # Calculando o produtório das somas acima estabelecidas
            weight_diss_prod = functools.reduce(
                operator.mul, weight_diss_sum1)

            # Calculado o somatório do denominador da equação à esquerda da igualdade
            weight_diss_sum2 = sum([u[i, k] * match(i, D[j], Gk[k])
                                    for i in range(N)])

            # Executando a divisão da fração à esquerda da equação
            l[k, j] = math.pow(weight_diss_prod, 1/P) / weight_diss_sum2

    return l

## Grau de pertinência

![Fórmula grau de pertinência](./img/membership_degree.png)

In [6]:
def compute_membership_degree(weight_matrix,
                              prototypes,
                              clusters_qtd,
                              dissimilarity_matrices,
                              elements_qtd,
                              m):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                elements_qtd: int
                    Quantidade de elementos da base de dados
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array NxK

    """
        

    K = clusters_qtd
    G = prototypes
    D = dissimilarity_matrices
    l = weight_matrix
    P = len(D)
    N = elements_qtd
    u = np.zeros((N, K))
    
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching

    def ratio(element, k, h):
        r = match(l, k, element, G, D) / match(l, h, element, G, D)
        return math.pow(r, 1/(m-1))

    for i in range(N):
        for k in range(K):
            outter_sum = sum([ratio(i, k, h) for h in range(K)])
            u[i, k] = 1/outter_sum

    return u

## Algoritmo completo
> Partitioning fuzzy K-medoids clustering algorithms with relevance weight for each dissimilarity matrix estimated locally

* Parametros: $K = 10; m = 1.6; T = 150; \epsilon = 10^{−10};$
* Devemos considerar a iniciarlizar do vetor de pesos como sendo 1, já que usamos a equação 9 (MFCMdd-RWL-P)

In [7]:
def random_prototypes(K, N, q, seed):
    elements = set(range(N))
    protos = []
    random.seed(seed)
    for k in range(K):
        protos.append(random.sample(elements, q))
        elements -= set(protos[-1])

    return protos


def executar_algoritmo(dissimilarity_matrices,
                       elements_qtd,
                       K=10,
                       m=1.6,
                       T=150,
                       epsilon=10e-10,
                       q=3, 
                       seed=13082020):

    D = dissimilarity_matrices
    N = elements_qtd
    P = len(D)

    last_lambda = np.ones((K, P))
    last_prototypes = random_prototypes(K, N, q, seed)
    last_membership_degree = None
    last_cost = None

    print("Passo 0")
    print("Calculando matriz de adequação inicial (u0)")
    u0 = compute_membership_degree(weight_matrix=last_lambda,
                                   prototypes=last_prototypes,
                                   clusters_qtd=K,
                                   dissimilarity_matrices=dissimilarity_matrices,
                                   elements_qtd=N,
                                   m=m)

    print("Calculando função de custo inicial (J0)")
    J0 = objective_function(clusters_qtd=K,
                            elements_qtd=N,
                            adequacy_criterion=u0,
                            m=m,
                            weight_matrix=last_lambda,
                            prototypes=last_prototypes,
                            dissimilarity_matrices=dissimilarity_matrices)
    
    last_membership_degree = u0
    last_cost = J0
    
    for t in range(1, T):
        print(f"Passo {t}/{T}")
        
        print(">> Calculando protótipos")
        new_prototypes = [get_prototypes(elements_qtd=N,
                                         q=q,
                                         m=m,
                                         s=1,
                                         cluster_number=k,
                                         adequacy_criterion=last_membership_degree,
                                         dissimilarity_matrices=D,
                                         weight_matrix=last_lambda) for k in range(K)]
        
        #print("new_prototypes.shape", new_prototypes)
        
        print(">> Calculando matriz de relevâncias")
        new_lambda = compute_relevance_weights(clusters_qtd=K,
                                               dissimilarity_matrices=D,
                                               prototypes=new_prototypes,
                                               elements_qtd=N,
                                               adequacy_criterion=last_membership_degree,
                                               m=m)

        print(">> Calculando grau de pertinência")
        new_degree = compute_membership_degree(weight_matrix=new_lambda,
                                               prototypes=new_prototypes,
                                               clusters_qtd=K,
                                               dissimilarity_matrices=dissimilarity_matrices,
                                               elements_qtd=N,
                                               m=m)

        print(">> Calculando função objetivo")
        new_cost = objective_function(clusters_qtd=K,
                                      elements_qtd=N,
                                      adequacy_criterion=new_degree,
                                      m=m,
                                      weight_matrix=new_lambda,
                                      prototypes=new_prototypes,
                                      dissimilarity_matrices=dissimilarity_matrices)

        last_prototypes = new_prototypes
        last_lambda = new_lambda
        last_membership_degree = new_degree
        print(">> Cost: ", new_cost)
        
        if abs(last_cost - new_cost) <= epsilon:
            last_cost = new_cost
            break
    
        last_cost = new_cost
        
    data = {
        "cost":last_cost,
        "membership_degree":last_membership_degree,
        "prototypes":last_prototypes,
        "weight_matrix":last_lambda 
    }
    return data

## Carregando Matrizes

In [8]:
def carregar_matrizes(data_path = None):
    data_path = data_path or "./data"
    data_path = os.path.abspath(data_path)
    
    FAC_FILE = os.path.join(data_path, "mfeat-fac-dissimilarity.npy")
    FOU_FILE = os.path.join(data_path, "mfeat-fou-dissimilarity.npy")
    KAR_FILE = os.path.join(data_path, "mfeat-kar-dissimilarity.npy")

    fac_dis = np.load(FAC_FILE)
    fou_dis = np.load(FOU_FILE)
    kar_dis = np.load(KAR_FILE)
    
    return fac_dis, fou_dis, kar_dis

## Executando algoritmo

In [9]:
def main():
    data_path = sys.argv[1] if len(sys.argv) == 2 else None
    
    fac_dis, fou_dis, kar_dis = carregar_matrizes(data_path)
    
    dissimilarity_matrices = [fac_dis, fou_dis, kar_dis]
    N = fac_dis.shape[0]
    data = executar_algoritmo(dissimilarity_matrices, N, q=3)
    
    print(data["prototypes"])
    return data

clustering_data = main()

Passo 0
Calculando matriz de adequação inicial (u0)
Calculando função de custo inicial (J0)
Passo 1/150
>> Calculando protótipos
>> Calculando matriz de relevâncias
>> Calculando grau de pertinência
>> Calculando função objetivo
>> Cost:  10939.879478157944
Passo 2/150
>> Calculando protótipos
>> Calculando matriz de relevâncias
>> Calculando grau de pertinência
>> Calculando função objetivo
>> Cost:  10939.866623836544
Passo 3/150
>> Calculando protótipos
>> Calculando matriz de relevâncias
>> Calculando grau de pertinência
>> Calculando função objetivo
>> Cost:  10939.866622965199
Passo 4/150
>> Calculando protótipos
>> Calculando matriz de relevâncias
>> Calculando grau de pertinência
>> Calculando função objetivo
>> Cost:  10939.86662296508
[array([ 568, 1748, 1797]), array([625, 616, 579]), array([1713,  884, 1666]), array([ 840, 1375, 1432]), array([1386,  118,  661]), array([643, 588,  59]), array([1112,  398,   42]), array([ 105, 1771, 1183]), array([ 990, 1236, 1055]), array([

## Partições hard

In [10]:
def get_hard_patitions(membership_degree):
    """
        membership_degree: numpy array of shape N x K
    """
    u = membership_degree
    members = []
    K = membership_degree.shape[1]
    
    # Obtendo o índice do grupo em que cada elemento possui maior valor de pertencimento
    element_index_membership = u.argsort(axis=1)[:, 0]
    
    for k in range(K):
        # Para cada grupo k, extrai quais dos elementos possui maior grau de pertencimento para ele
        memb = np.where(element_index_membership == k)[0]
        members.append(memb)
        
    return members

membership_degree = clustering_data["membership_degree"]
hard_members = get_hard_patitions(membership_degree)

for group_number, group in enumerate(hard_members):
    print(group_number, group, len(group))

print("Total de elementos somados:",  sum([len(g) for g in hard_members]))

0 [ 709  800  803  838  843  846  860  877  881  888  910  911  915  916
  941  954  957  963  974 1120 1841 1845 1882] 23
1 [ 205  219  253  271  290  307  323  327  343  357  805  808  811  821
  825  842  844  845  849  850  856  858  864  865  868  872  875  883
  885  886  889  891  893  895  897  898  906  908  918  920  922  924
  926  927  930  934  937  938  944  945  956  964  965  971  975  980
  988  990  992  999 1074 1200 1202 1205 1208 1215 1216 1223 1231 1236
 1246 1282 1291 1302 1310 1314 1317 1323 1326 1328 1331 1334 1336 1337
 1354 1356 1359 1360 1393 1394 1667 1696 1717 1729] 94
2 [ 446  449  463  478  523  539  549  555  597  606  616  619  622  627
  629  633  635  653  665  674  695  697  714  720  729  750  761  769
  772  779  780  781  950  987 1001 1019 1032 1072 1075 1081 1084 1109
 1136 1142 1144 1152 1154 1156 1157 1165 1170 1177 1257 1402 1405 1412
 1414 1418 1419 1423 1425 1427 1428 1429 1432 1435 1441 1442 1444 1449
 1452 1457 1460 1465 1474 1476 1478 1

## Funções de validação do cluster

In [11]:
def calc_partition_coefficient(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.power(membership_degree, 2)
    return membership_degree.sum(axis = 1).sum()/n

def calc_modified_partition_coefficient(membership_degree):
    c = membership_degree.shape[1]
    vpc = calc_partition_coefficient(membership_degree)
    return 1 - (c/(c-1))*(1-vpc)

def calc_partition_entropy(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.log10(membership_degree) * membership_degree
    return -membership_degree.sum(axis = 1).sum()/n

partition_coeficient = calc_modified_partition_coefficient(membership_degree)
partition_entropy = calc_partition_entropy(membership_degree)

print("Modified partition coefficient:", partition_coeficient)
print("Parition Entropy:", partition_entropy)


Modified partition coefficient: 0.0017040598130578788
Parition Entropy: 0.9967684860445947
